In [42]:
from PIL import Image

# Open an image file
image = Image.open('81.png')
# Display the image (optional)
image.show()

# You can also access the image's size and format
print(f"Image size: {image.size}")
print(f"Image format: {image.format}")

Image size: (1508, 420)
Image format: PNG


In [73]:
from PIL import Image

# Open an image file
with Image.open('81.png') as img:
    # Resize the image to a new width and height
    new_size = (img.width*2, img.height //6)  # Decrease size by half
    resized_img = img.resize(new_size)

    # Convert to RGB mode to save as JPEG (since JPEG doesn't support RGBA)
    resized_img = resized_img.convert("RGB")

    # Save the resized image
    resized_img.save('resized_image.jpg')

print("Image resized successfully!")


Image resized successfully!


In [74]:
from PIL import Image

# Open an image file
image = Image.open('resized_image.jpg')
# Display the image (optional)
image.show()

# You can also access the image's size and format
print(f"Image size: {image.size}")
print(f"Image format: {image.format}")

Image size: (3016, 70)
Image format: JPEG


In [76]:
import cv2
import numpy as np

# Load the image
image_path = 'resized_image.jpg'
img = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply binary threshold to get a binary image
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# If no contours are found, exit the process
if not contours:
    print("No contours found!")
    exit()

# Assuming we want the largest contour (the highlighted part)
largest_contour = max(contours, key=cv2.contourArea)

# Find the bounding rectangle for the largest contour
rect = cv2.minAreaRect(largest_contour)
box_points = cv2.boxPoints(rect)
box_points = np.int32(box_points)  # Corrected


# Crop the image based on the bounding box of the largest contour
# Create a mask of the bounding box area
mask = np.zeros_like(gray)
cv2.fillPoly(mask, [box_points], 255)

# Apply the mask to the original image to extract the ROI
roi = cv2.bitwise_and(img, img, mask=mask)

# Calculate the length and width of the bounding rectangle (in pixels)
length_pixels = max(np.linalg.norm(box_points[0] - box_points[1]), np.linalg.norm(box_points[1] - box_points[2]))
width_pixels = min(np.linalg.norm(box_points[0] - box_points[1]), np.linalg.norm(box_points[1] - box_points[2]))

# Calculate the area of the rectangle (in pixels)
area_pixels = length_pixels * width_pixels

# Assume the DPI of the image is 640 (this should be known or provided)
dpi = 640

# Convert length and width to millimeters
length_mm = (length_pixels / dpi) * 25.4
width_mm = (width_pixels / dpi) * 25.4

# Convert area to square millimeters
area_mm2 = (area_pixels / dpi**2) * 25.4**2

# Show the cropped image (highlighted part)

cv2.imwrite('highlighted_part.jpg', roi)
print("Highlighted part saved as 'highlighted_part.jpg'")
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Print the results in millimeters
print(f"Area: {area_mm2:.2f} mm²")
print(f"Length: {length_mm:.2f} mm")
print(f"Width: {width_mm:.2f} mm")


Highlighted part saved as 'highlighted_part.jpg'
Area: 0.69 mm²
Length: 3.45 mm
Width: 0.20 mm
